# Lista 08 - Comparando Regressões

# Exercício 01:

Analise o desempenho do kNN e de uma Regressão Linear Regularizada para **pelo menos um** dos conjuntos de dados disponível na [seção de regressão linear múltipla](http://college.cengage.com/mathematics/brase/understandable_statistics/7e/students/datasets/mlr/frames/frame.html) da página do *Livro Understandable Statistics* de Charles Brase e Corrinne Brase. Para o conjunto de dados que escolheu, execute a regressão linear múltipla para explicar o fator $X1$ dos dados ([ver descrição de um dos conjuntos](http://college.cengage.com/mathematics/brase/understandable_statistics/7e/students/datasets/mlr/frames/frame.html)) a partir dos outros fatores. 

Para a questão, faça as seguintes tarefas:

* Realize treino, validação e teste
* Compare as métricas no teste
* Diferente da lista anterior, reporte o erro quadrado médio no conjunto de teste.

In [1]:
# YOUR CODE HERE
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [2]:
#importação dos dados
df = pd.read_excel('./mlr10.xls')
df.head()

*** No CODEPAGE record, no encoding_override: will use 'ascii'


,X1,X2,X3,X4,X5,X6,X7
0,6.9,1.8,30.2,58.3,27.3,84.9,-14.2
1,8.4,28.5,38.8,87.5,39.8,172.6,-34.1
2,5.7,7.8,31.7,83.5,26.0,154.2,-15.8
3,7.4,2.3,24.2,14.2,29.4,35.2,-13.9
4,8.5,-0.7,28.1,46.7,26.6,69.2,-13.9


In [3]:
X = df.loc[:, df.columns != 'X1'].values
y = df[['X1']].values.ravel()

X_train_t, X_test, y_train_t, y_test = train_test_split(X, y, test_size=0.3)

y_ = np.around(y.ravel(),0)

In [4]:
from sklearn.model_selection import GridSearchCV

knn = KNeighborsClassifier()
param_grid = {'n_neighbors': np.arange(1, 25)}
knn_gscv = GridSearchCV(knn, param_grid, cv=5)
knn_gscv.fit(X, y_)
knn_gscv.best_params_

/usr/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/usr/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'n_neighbors': 4}

In [5]:
y_train_t_int = np.around(y_train_t.ravel(),0)

model = KNeighborsClassifier(n_neighbors=knn_gscv.best_params_.get('n_neighbors'))
model.fit(X_train_t, y_train_t_int.ravel())
y_result_test = model.predict(X_test)

In [6]:
print('Erros quadráticos: ', ((y_result_test - y_test)**2).mean())


Erros quadráticos:  23.672857142857143


In [7]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler

#Regressão Linear Regularizada

boston = load_boston()
X = boston.data
y = boston.target

scaler = StandardScaler()
X_std = scaler.fit_transform(X)

In [8]:
regr_cv = RidgeCV(alphas=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 2, 3, 4, 5, 6, 7, 8, 9, 10])

In [9]:
model_cv = regr_cv.fit(X_std, y)

In [10]:
model_cv.alpha_

5.0

In [11]:
model = Ridge(alpha=model_cv.alpha_, fit_intercept=True).fit(X_train_t, y_train_t)
y_rid_test = model.predict(X_test)

In [12]:
print('Erros quadráticos: ', ((y_rid_test - y_test)**2).mean())

Erros quadráticos:  9.670627704512594


Explique e discuta sobre os resultados encontrados no campo abaixo.

YOUR ANSWER HERE

Conforme observado pelos erros quadráticos, os métodos obtiveram bons resultados para as regressões, sobretudo considerando as dimensões reduzidas do dataset disponibilizado.

Comparando os métodos entre si, pode-se notar que a Regressão Linear Regularizada obteve um desempenho melhor que o KNN (9.67 vs 23.6 considerando os erros quadráticos). A razão dessa diferença diz respeito à natureza do método kNN, que aceita apenas classes, o que nos obrigou a transformar todos os números racionais em inteiros, perdendo precisão. 

Dessa forma, observa-se a importância da reflexão a respeito da escolha do modelo que melhor adequa aos dados disponíveis, já que, mesmo que sejam poucos dados (como é o caso atual), uma boa escolha pode gerar uma boa regressão, conforme visto para a Regressão Linear Regularizada acima.